# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
from openai import OpenAI
from api_key_loader import ApiKeyLoader
import gradio as gr

In [2]:
# Initialization
keys = ApiKeyLoader()

In [3]:
# Create LLM client
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [4]:
from tool_box import ToolBox

In [5]:
tool_box = ToolBox(keys)

In [6]:
system_message = "You are a helpful assistant for an Travel Agency called TravelAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [7]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tool_box.tools)
    image = None

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tools_response, image = tool_box.apply(message)
        messages.append(message)
        messages.extend(tools_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    return history, image

In [ ]:
resp, image = chat([{"role": "user", "content": "Show me Milan Airports on the map"}])

In [9]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
from travel_api import TravelAPI
trave_agent = TravelAPI(keys.get("client_id"), keys.get("client_secret"))
airports = trave_agent.get_airport('Milan')
print(airports)

In [8]:
resp, image = chat([{"role": "user", "content": "Give me a boarding pass from MXP to LHR for 1 Nov 2025 for Sergei Sevriugin"}])